In [ ]:
# ============================================================================
# DATA PREPROCESSING
# ============================================================================
import numpy as np
import torch
from torch.utils.data import TensorDataset
from preprocessing import run_preprocessing
from windows import build_windows

# Preprocess Data
df_train, df_val, train_targets, val_targets = run_preprocessing()

# Hyperparameters
WINDOW_SIZE = 300
STRIDE = 150
BATCH_SIZE = 64

# Build sequences - returns 3D arrays (samples, timesteps, features)
X_train, y_train, _ = build_windows(df_train, train_targets, WINDOW_SIZE, STRIDE, feature="3d")
X_val, y_val, _ = build_windows(df_val, val_targets, WINDOW_SIZE, STRIDE, feature="3d")

print(f"Training set shape: {X_train.shape}, {y_train.shape}")
print(f"Validation set shape: {X_val.shape}, {y_val.shape}")

# Compute class distribution
class_counts = np.bincount(y_train.astype(int))
print("\n📊 Class distribution in training set:")
for cls, count in enumerate(class_counts):
    print(f"  Class {cls}: {count} samples ({count/len(y_train)*100:.1f}%)")

# ============================================================================
# WEIGHTED SAMPLING (to handle class imbalance)
# ============================================================================
# Calculate inverse-frequency weights per class
class_weights = 1.0 / class_counts
class_weights = class_weights / np.sum(class_weights)  # Normalize weights
print("\n⚖️  Sample weights (inverse frequency):")
for cls, weight in enumerate(class_weights):
    print(f"  Class {cls}: {weight:.4f}")

# Assign a weight to each sample based on its class
sample_weights = class_weights[y_train.astype(int)]
sample_weights = torch.from_numpy(sample_weights).double()

# Create WeightedRandomSampler (oversamples minority classes)
'''
sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)
'''
# ============================================================================
# CREATE DATALOADERS
# ============================================================================
# Convert to PyTorch datasets
'''
train_ds = TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).long())
val_ds = TensorDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).long())

# Create data loaders
train_loader = make_loader(train_ds, batch_size=BATCH_SIZE, shuffle=False, drop_last=True, sampler=sampler)
val_loader = make_loader(val_ds, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)
'''
# CREATE DATALOADERS
train_ds = TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).long())
val_ds = TensorDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).long())

# Create data loaders - SOLO class weights, niente sampler
train_loader = make_loader(
    train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,      # ← ORA TRUE
    drop_last=True
)

val_loader = make_loader(
    val_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    drop_last=False
)


# Store metadata for model creation
input_shape = X_train.shape
num_classes = len(np.unique(y_train))

print(f"\n✅ DataLoaders created")
print(f"   Training batches: {len(train_loader)}")
print(f"   Validation batches: {len(val_loader)}")
print(f"   Input shape: {input_shape}")
print(f"   Number of classes: {num_classes}")

In [ ]:
import torch.nn as nn
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class CNN1DClassifier_bidirectional(nn.Module):
    def __init__(
        self,
        input_size,
        num_classes,
        hidden_size,
        dropout=0.4,
        bidirectional=False,
        rnn_type='GRU',     # 'RNN', 'LSTM', 'GRU'
        num_layers=1
    ):
        super().__init__()

        # ====== CNN BLOCK ======
        self.conv1 = nn.Conv1d(input_size, 64, kernel_size=5, padding=2)
        self.bn1   = nn.BatchNorm1d(64)

        self.conv2 = nn.Conv1d(64, 128, kernel_size=5, padding=2)
        self.bn2   = nn.BatchNorm1d(128)

        self.conv3 = nn.Conv1d(128, 256, kernel_size=3, padding=1)
        self.bn3   = nn.BatchNorm1d(256)

        self.relu    = nn.ReLU()
        self.pool    = nn.MaxPool1d(2)
        self.dropout = nn.Dropout(dropout)

        # ====== RNN BLOCK ======
        self.rnn_type     = rnn_type
        self.num_layers   = num_layers
        self.hidden_size  = hidden_size
        self.bidirectional = bidirectional

        rnn_map = {
            'RNN': nn.RNN,
            'LSTM': nn.LSTM,
            'GRU': nn.GRU
        }
        if rnn_type not in rnn_map:
            raise ValueError("rnn_type must be 'RNN', 'LSTM', or 'GRU'")

        rnn_module = rnn_map[rnn_type]

        # Dropout solo tra i layer dell'RNN (se num_layers > 1)
        rnn_dropout = dropout if num_layers > 1 else 0.0

        # L'RNN prende in input i canali finali della CNN: 256
        self.rnn = rnn_module(
            input_size=256,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=bidirectional,
            dropout=rnn_dropout
        )

        # ====== CLASSIFIER ======
        if self.bidirectional:
            classifier_input_size = hidden_size * 2  # concat fwd + bwd
        else:
            classifier_input_size = hidden_size

        self.fc = nn.Sequential(
            nn.Linear(classifier_input_size, 128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        # x: (batch, seq_len, features)
        # CNN vuole (batch, channels, seq_len)
        x = x.transpose(1, 2)   # -> (batch, features, seq_len)

        # ----- CNN -----
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.dropout(x)

        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.dropout(x)

        x = self.relu(self.bn3(self.conv3(x)))
        # x: (batch, 256, seq_len_cnn)

        # ----- RNN -----
        # RNN vuole (batch, seq_len, features)
        x_rnn_in = x.transpose(1, 2)  # -> (batch, seq_len_cnn, 256)

        rnn_out, hidden = self.rnn(x_rnn_in)
        # rnn_out: (batch, seq_len_cnn, hidden_size * num_directions)
        # hidden:  (num_layers * num_directions, batch, hidden_size)
        # per LSTM: hidden = (h_n, c_n), ci serve solo h_n
        if self.rnn_type == 'LSTM':
            hidden = hidden[0]

        if self.bidirectional:
            # (num_layers, 2, batch, hidden_size)
            hidden = hidden.view(self.num_layers, 2, -1, self.hidden_size)
            # ultimi stati fwd e bwd dell’ultima layer
            hidden_to_classify = torch.cat(
                [hidden[-1, 0, :, :], hidden[-1, 1, :, :]],
                dim=1
            )  # -> (batch, hidden_size * 2)
        else:
            hidden_to_classify = hidden[-1]  # (batch, hidden_size)

        logits = self.fc(hidden_to_classify)
        return logits

model = CNN1DClassifier_bidirectional(
    input_size=X_train.shape[-1],
    num_classes=3,
    hidden_size=256,
    dropout=0.3,
    bidirectional=True
).to(device)
print("CNN_biderectional Model created with stronger regularization")
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"\n🔧 Model: {model.__class__.__name__}")
print(f" Parameters: {sum(p.numel() for p in model.parameters()):,}")

# ----------------------------------------------------------------------------
# 2. LOSS FUNCTION (with class weights for imbalance)
# ----------------------------------------------------------------------------
# Calculate class weights: inverse frequency with normalization
'''
train_class_counts = np.bincount(y_train.astype(int))
class_weights_loss = len(y_train) / (len(train_class_counts) * train_class_counts)
class_weights_loss = torch.tensor(class_weights_loss, dtype=torch.float32).to(device)

print(f"\n⚖️  Loss weights (amplifies gradients for minority classes):")
for cls, weight in enumerate(class_weights_loss):
    print(f"  Class {cls}: {weight:.4f}x")

criterion = nn.CrossEntropyLoss(weight=class_weights_loss)
'''

# ----------------------------------------------------------------------------
# 2. LOSS FUNCTION (with class weights for imbalance) - SOLO CLASS WEIGHTS
# ----------------------------------------------------------------------------
class_counts = np.bincount(y_train.astype(int))

# Pesi inversamente proporzionali alla frequenza
class_weights_loss = 1.0 / class_counts
class_weights_loss = class_weights_loss / class_weights_loss.sum()  # opzionale, normalizza

class_weights_loss = torch.tensor(class_weights_loss, dtype=torch.float32).to(device)

print(f"\n⚖️  Loss weights (amplifies gradients for minority classes):")
for cls, weight in enumerate(class_weights_loss):
    print(f"  Class {cls}: {weight:.4f}x")

criterion = nn.CrossEntropyLoss(weight=class_weights_loss)


# ----------------------------------------------------------------------------
# 3. OPTIMIZER
# ----------------------------------------------------------------------------
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=5e-4,           # Learning rate
    weight_decay=1e-4  # L2 regularization
)

# Alternative optimizers:
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

# ----------------------------------------------------------------------------
# 4. GRADIENT SCALER (for mixed precision training)
# ----------------------------------------------------------------------------
scaler = torch.amp.GradScaler(enabled=(device.type == 'cuda'))

In [ ]:
# ============================================================================
# TRAINING
# ============================================================================
from model_logic import fit
print(f"\n{'='*70}")
print(f"Training {model.__class__.__name__}...")
print(f"{'='*70}")

_, history = fit(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=1500,
    criterion=criterion,
    optimizer=optimizer,
    scaler=scaler,
    device=device,
    writer=None,
    verbose=3,
    experiment_name="model_training",
    patience=100,       # Set > 0 for early stopping
    l1_lambda=0,      # L1 regularization
    l2_lambda=0       # L2 regularization (or use weight_decay in optimizer)
)

# ============================================================================
# RESULTS
# ============================================================================
print("\n" + "=" * 70)
print("📊 TRAINING RESULTS:")
print("=" * 70)
print(f"  Initial val F1: {history['val_f1'][0]:.4f}")
print(f"  Final val F1:   {history['val_f1'][-1]:.4f}")
print(f"  Best val F1:    {max(history['val_f1']):.4f}")
print(f"  Improvement:    {max(history['val_f1']) - history['val_f1'][0]:+.4f}")

# Per-class predictions
from sklearn.metrics import classification_report
model.eval()
val_preds = []
val_true = []
with torch.no_grad():
    for inputs, targets in val_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        preds = outputs.argmax(dim=1)
        val_preds.extend(preds.cpu().numpy())
        val_true.extend(targets.cpu().numpy())

print("\n" + "=" * 70)
print("📈 CLASSIFICATION REPORT:")
print("=" * 70)
print(classification_report(
    val_true, val_preds,
    target_names=['no_pain', 'low_pain', 'high_pain'],
    digits=4
))